In [47]:
import pandas as pd 
import numpy as np 

from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import BoxScoreAdvancedV3
from nba_api.stats.endpoints.teamestimatedmetrics import TeamEstimatedMetrics
from nba_api.stats.library.parameters import LeagueID

In [48]:
# Get team ID for the Toronto Raptors
team_dict = teams.get_teams()
raptors = [team for team in team_dict if team['abbreviation'] == 'TOR'][0]
raptors_id = raptors['id']

# Get team ID for matchup
opp_abbreviation = 'BOS'
opponent = [team for team in team_dict if team['abbreviation'] == opp_abbreviation][0]
opp_id = opponent['id']

# Set season
chosen_season='2023-24'

# Set game type
game_type = 'Regular Season'

In [52]:
# Get all of the Raptors games
raptors_game_log = teamgamelog.TeamGameLog(
    team_id=raptors_id,
    season=chosen_season,
    season_type_all_star = game_type,
    league_id_nullable = ''
)

# Access the game log data
raptors_games_data = raptors_game_log.team_game_log.get_dict()['data']

# Create a DataFrame with labeled columns
columns = raptors_game_log.expected_data['TeamGameLog']
raptors_df = pd.DataFrame(raptors_games_data, columns=columns)


# Print the head of the DataFrame
print(raptors_df.head())

      Team_ID     Game_ID     GAME_DATE      MATCHUP WL   W   L  W_PCT  MIN  \
0  1610612761  0022300923  MAR 09, 2024    TOR @ POR  L  23  41  0.359  265   
1  1610612761  0022300905  MAR 07, 2024    TOR @ PHX  L  23  40  0.365  240   
2  1610612761  0022300890  MAR 05, 2024  TOR vs. NOP  L  23  39  0.371  240   
3  1610612761  0022300875  MAR 03, 2024  TOR vs. CHA  W  23  38  0.377  240   
4  1610612761  0022300860  MAR 01, 2024  TOR vs. GSW  L  22  38  0.367  240   

   FGM  ...  FT_PCT  OREB  DREB  REB  AST  STL  BLK  TOV  PF  PTS  
0   43  ...   0.842     9    34   43   29    9    6   20  18  118  
1   43  ...   0.733     9    27   36   30    5    2    9  20  113  
2   36  ...   0.591    11    34   45   21    6    7   11  11   98  
3   42  ...   0.778    11    34   45   27    8    6   14  16  111  
4   41  ...   0.647    11    36   47   31    9    2   14  14  105  

[5 rows x 27 columns]


In [53]:
# List of columns to drop
columns_to_drop = ['Team_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'W', 'L', 'W_PCT', 'MIN']

# Drop the specified columns
raptors_df = raptors_df.drop(columns=columns_to_drop)

# Rename column to merge later
raptors_df = raptors_df.rename(columns={'Game_ID': 'gameId'})

print(raptors_df.head())

       gameId  FGM  FGA  FG_PCT  FG3M  FG3A  FG3_PCT  FTM  FTA  FT_PCT  OREB  \
0  0022300923   43   91   0.473    16    43    0.372   16   19   0.842     9   
1  0022300905   43   94   0.457    16    44    0.364   11   15   0.733     9   
2  0022300890   36   93   0.387    13    44    0.295   13   22   0.591    11   
3  0022300875   42   91   0.462    13    38    0.342   14   18   0.778    11   
4  0022300860   41   96   0.427    12    45    0.267   11   17   0.647    11   

   DREB  REB  AST  STL  BLK  TOV  PF  PTS  
0    34   43   29    9    6   20  18  118  
1    27   36   30    5    2    9  20  113  
2    34   45   21    6    7   11  11   98  
3    34   45   27    8    6   14  16  111  
4    36   47   31    9    2   14  14  105  


In [55]:
# Function to get advanced stats for a specific game
def get_advanced_stats(game_id):
    boxscore_advanced = BoxScoreAdvancedV3(game_id)
    team_stats = boxscore_advanced.team_stats.get_data_frame()
    return team_stats

# Function to iterate through games and create a DataFrame
def create_advanced_stats_dataframe(game_ids):
    team_stats_list = []

    for game_id in game_ids:
        team_stats = get_advanced_stats(game_id)
        team_stats_list.append(team_stats)

    all_team_stats = pd.concat(team_stats_list, ignore_index=True)

    return all_team_stats

# Extract the list of game IDs from the 'Game_ID' column of 'raptors_df'
game_ids_list = raptors_df['gameId'].tolist()

team_adv_stats_df = create_advanced_stats_dataframe(game_ids_list)


In [56]:
print(team_adv_stats_df.head())

       gameId      teamId  teamCity       teamName teamTricode teamSlug  \
0  0022300923  1610612757  Portland  Trail Blazers         POR  blazers   
1  0022300923  1610612761   Toronto        Raptors         TOR  raptors   
2  0022300905  1610612756   Phoenix           Suns         PHX     suns   
3  0022300905  1610612761   Toronto        Raptors         TOR  raptors   
4  0022300890  1610612761   Toronto        Raptors         TOR  raptors   

  minutes  estimatedOffensiveRating  offensiveRating  \
0  265:00                     120.1            119.6   
1  265:00                     106.9            111.3   
2  240:00                     119.0            121.2   
3  240:00                     112.3            114.1   
4  240:00                      94.5             93.3   

   estimatedDefensiveRating  ...  turnoverRatio  effectiveFieldGoalPercentage  \
0                     106.9  ...           14.0                         0.553   
1                     120.1  ...           18.9   

In [57]:
# Extract the raptors advanced stats
raptors_adv_team_stats = team_adv_stats_df[team_adv_stats_df['teamTricode'] == 'TOR']

# Drop columns
columns_to_drop = ['teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'minutes']
raptors_adv_team_stats = raptors_adv_team_stats.drop(columns=columns_to_drop)

# Rename columns
raptors_adv_team_stats = raptors_adv_team_stats.rename(columns=lambda x: 'TOR_' + x if x != 'gameId' else x)
print(raptors_adv_team_stats.head())

       gameId  TOR_estimatedOffensiveRating  TOR_offensiveRating  \
1  0022300923                         106.9                111.3   
3  0022300905                         112.3                114.1   
4  0022300890                          94.5                 93.3   
6  0022300875                         106.8                106.7   
8  0022300860                          98.6                101.0   

   TOR_estimatedDefensiveRating  TOR_defensiveRating  TOR_estimatedNetRating  \
1                         120.1                119.6                   -13.2   
3                         119.0                121.2                    -6.7   
4                         133.9                135.0                   -39.3   
6                         100.9                101.9                     5.9   
8                         114.4                114.3                   -15.8   

   TOR_netRating  TOR_assistPercentage  TOR_assistToTurnover  TOR_assistRatio  \
1           -8.3             

In [58]:
# Extract the opponents advanced stats
opposing_adv_team_stats = team_adv_stats_df[team_adv_stats_df['teamTricode'] != 'TOR']

# Drop columns from 'opposing_team_stats'
columns_to_drop = ['teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'minutes']
opposing_adv_team_stats = opposing_adv_team_stats.drop(columns=columns_to_drop)

opposing_adv_team_stats = opposing_adv_team_stats.rename(columns=lambda x: 'OPP_' + x if x != 'gameId' else x)
print(opposing_adv_team_stats.head())

       gameId  OPP_estimatedOffensiveRating  OPP_offensiveRating  \
0  0022300923                         120.1                119.6   
2  0022300905                         119.0                121.2   
5  0022300890                         133.9                135.0   
7  0022300875                         100.9                101.9   
9  0022300860                         114.4                114.3   

   OPP_estimatedDefensiveRating  OPP_defensiveRating  OPP_estimatedNetRating  \
0                         106.9                111.3                    13.2   
2                         112.3                114.1                     6.7   
5                          94.5                 93.3                    39.3   
7                         106.8                106.7                    -5.9   
9                          98.6                101.0                    15.8   

   OPP_netRating  OPP_assistPercentage  OPP_assistToTurnover  OPP_assistRatio  \
0            8.3             

In [59]:
# Merge DataFrames based on the common 'gameId' column
combined_adv_stats = pd.merge(raptors_adv_team_stats, opposing_adv_team_stats, on='gameId')
print(combined_adv_stats)


        gameId  TOR_estimatedOffensiveRating  TOR_offensiveRating  \
0   0022300923                         106.9                111.3   
1   0022300905                         112.3                114.1   
2   0022300890                          94.5                 93.3   
3   0022300875                         106.8                106.7   
4   0022300860                          98.6                101.0   
..         ...                           ...                  ...   
59  0022300116                         129.8                134.0   
60  0022300105                          92.3                 97.8   
61  0022300092                         112.2                116.3   
62  0022300082                          93.6                 94.5   
63  0022300069                          94.1                 96.0   

    TOR_estimatedDefensiveRating  TOR_defensiveRating  TOR_estimatedNetRating  \
0                          120.1                119.6                   -13.2   
1        

In [1]:
game_data = pd.merge(raptors_df, combined_adv_stats, on='gameId')
game_data = game_data.drop(columns='gameId')
print(game_data)

game_data.to_csv('game_data.csv', index=False)


NameError: name 'pd' is not defined